# Consolidate books dataset with information ffrom ratings dataset

## 0. Set up

Libraries

In [ ]:
#pip install -r requirements.txt

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/981.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/981.5 kB ? eta -:--:--
     - ----------------------------------- 30.7/981.5 kB 163.8 kB/s eta 0:00:06
     - ----------------------------------- 30.7/981.5 kB 163.8 kB/s eta 0:00:06
     - ----------------------------------- 30.7/981.5 kB 163.8 kB/s eta 0:00:06
     - ----------------------------------- 30.7/981.5 kB 163.8 kB/s eta 0:00:06
     - ----------------------------------- 30.7/981.5 kB 163.8 kB/s eta 0:00:06
     - ------------------------------------ 41.0/981.5 kB 89.3 kB/s eta 0:00:11
     -- ---------------------------------- 61.4/981.5 kB 142.6 kB/s eta 0:00:07
     -- ---------------------------------- 71.7/981.5 kB 140.6 kB/s eta 0:00:07
     ---- ------------------------------- 122.9/981.5 kB 232.7 kB/s eta 0:00:04
     -------- --------------------------- 235.5/981.5 kB 41

In [267]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [327]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import re
import datetime

# EDA functions
from EDA_functions import ratings_cleaning_steps
from EDA_functions import folders_set_up
from EDA_functions import books_cleaning_steps
from NLP_preprocessing import identify_language

# import requests
# from bs4 import BeautifulSoup
# from PIL import Image
# from io import BytesIO

Project folder

In [269]:
github_folder, input_folder, output_folder = folders_set_up.generate_folders()

## 1. Import data

In [270]:
ratings_info = pd.read_csv(
    os.path.join(output_folder, "book_ratings_info.csv")
)
ratings_info.shape

(212404, 8)

In [271]:
books = pd.read_csv(
    os.path.join(input_folder, "books_data_with_index.csv")
)
books.shape

(212404, 11)

## 2. Merge datasets

In [292]:
full_df = pd.merge(
    books,
    ratings_info,
    on = 'Title',
    how = 'inner'
)

In [293]:
full_df.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount,index,reviews number,average rating,median rating,reviews with missing price,average price,median price,percentage missing prices
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN,0,1,4.000000,4.0,1.0,NaN,NaN,1.0
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN,1,9,4.555556,5.0,9.0,NaN,NaN,1.0
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN,2,4,5.000000,5.0,0.0,19.40,19.40,0.0
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN,3,32,3.718750,5.0,0.0,10.95,10.95,0.0
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN,4,1,5.000000,5.0,0.0,39.95,39.95,0.0


## 3. Clean dataset

### 3.1 Published date
This variable presents different issues:
- Date is recorded as yyyy or dd-mm-yyyy format. Therefore it is not possible to directly convert Published date into a datetime.
- Some entries contain unclear year, such as "19??"
- Some dates are missing

Below we first apply a function which converts into dates based on the format (yyyy, yyyy-mm-dd or yyyy-mm-dd). Missing values are dropped.

#### 3.1.1 Explore column

In [294]:
print(f"Number of books with NA date: {full_df['publishedDate'].isna().sum():,}")

Number of books with NA date: 25,305


In [295]:
print("Number of books with unclear date (e.g. 19??):")
print(full_df['publishedDate'].astype(str).str.contains('\?', na=False).sum())

Number of books with unclear date (e.g. 19??):
128


In [296]:
# Inspect date for non missing values
full_df[~full_df['publishedDate'].isna()]['publishedDate'].sort_values()

157800     101-01-01
205865     101-01-01
160025     101-01-01
117218     101-01-01
56069      101-01-01
             ...    
96585           2025
98662     2030-12-31
191818          20??
63779           20??
125944          20??
Name: publishedDate, Length: 187099, dtype: object

#### 3.1.2 Apply function for extract year

In [297]:
# Apply function
full_df['date'] = full_df['publishedDate'].apply(books_cleaning_steps.datetime_conversion)

In [298]:
# Explore result
full_df[['publishedDate', 'date']].sort_values(by = 'publishedDate', ascending=False)

,publishedDate,date
63779,20??,NaT
191818,20??,NaT
125944,20??,NaT
98662,2030-12-31,2030-12-31
146995,2025,2025-01-01
...,...,...
212388,NaN,NaT
212391,NaN,NaT
212393,NaN,NaT
212395,NaN,NaT


#### 3.1.3 Remove missing values

In [305]:
# Remove missing values
#full_df = ratings_cleaning_steps.remove_na_titles(full_df, 'date')
#full_df[['publishedDate', 'date']].sort_values(by = 'publishedDate', ascending=False)

In [306]:
#print(f"Books after removing entries missing or unclear published dates {full_df.shape[0]:,}")

### 3.2 Description
Get description language

In [328]:
full_df["description_language"] = full_df["description"].map(identify_language, na_action='ignore')

141755

## 4. Checking dataset size based on cleaning approach 

In [340]:
min_number_of_reviews = 20

print(f"Books with English description: {full_df[full_df['description_language'] == 'English'].shape[0]:,}")
print(f"Books with a date: {full_df[~full_df['date'].isna()].shape[0]:,}")
print(f"Books with a price: {full_df[~full_df['average price'].isna()].shape[0]:,}")
print(f"Books with at least {min_number_of_reviews} reviews: {full_df[full_df['reviews number']>=min_number_of_reviews].shape[0]:,}")
print("\n")

print(f"Books with English description, a date and price: {full_df[(full_df['description_language'] == 'English') & (~full_df['average price'].isna()) & (~full_df['date'].isna())].shape[0]:,}")
print(f"Books with English description, a date and at least {min_number_of_reviews} reviews: {full_df[(full_df['description_language'] == 'English') & (~full_df['date'].isna()) & (full_df['reviews number']>=min_number_of_reviews)].shape[0]:,}")
print(f"Books with English description, a price and at least {min_number_of_reviews} reviews: {full_df[(full_df['description_language'] == 'English') & (~full_df['average price'].isna()) & (full_df['reviews number']>=min_number_of_reviews)].shape[0]:,}")
print("\n")

print(f"Books satisfying all criterias above: {full_df[(full_df['description_language'] == 'English') & (~full_df['average price'].isna()) & (~full_df['date'].isna()) & (full_df['reviews number']>=min_number_of_reviews)].shape[0]:,}")

Books with English description: 141,755
Books with a date: 186,728
Books with a price: 48,419
Books with at least 20 reviews: 22,432


Books with English description, a date and price: 37,534
Books with English description, a date and at least 20 reviews: 17,603
Books with English description, a price and at least 20 reviews: 4,136


Books satisfying all criterias above: 4,126
